# User Engagement Dataset

## Imports

In [1]:
import json
from datetime import datetime
import pandas as pd
import matplotlib as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, roc_auc_score, classification_report, accuracy_score, confusion_matrix 

## Data wrangling and EDA

In [2]:
# Open file
file_path = '/Users/pandabear/springboard/Takehomeassignment_2/'
takehome_users_df = pd.read_csv(file_path + 'takehome_users.csv', encoding='latin1')
takehome_user_engagement_df = pd.read_csv(file_path + 'takehome_user_engagement.csv', encoding='latin1')

In [3]:
takehome_users_df.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [4]:
takehome_users_df.shape

(12000, 10)

In [5]:
takehome_user_engagement_df.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [6]:
takehome_users_df.dtypes

object_id                       int64
creation_time                  object
name                           object
email                          object
creation_source                object
last_session_creation_time    float64
opted_in_to_mailing_list        int64
enabled_for_marketing_drip      int64
org_id                          int64
invited_by_user_id            float64
dtype: object

In [7]:
takehome_users_df.isna().sum()

object_id                        0
creation_time                    0
name                             0
email                            0
creation_source                  0
last_session_creation_time    3177
opted_in_to_mailing_list         0
enabled_for_marketing_drip       0
org_id                           0
invited_by_user_id            5583
dtype: int64

In [8]:
# Drop rows where there is no last session time (this means they have not logged in yet)
takehome_users_df.dropna(subset='last_session_creation_time', inplace=True)

In [9]:
# fill na for invited_by_user_id with 0
takehome_users_df.fillna(value=0, inplace=True)

In [10]:
takehome_users_df.isna().sum()

object_id                     0
creation_time                 0
name                          0
email                         0
creation_source               0
last_session_creation_time    0
opted_in_to_mailing_list      0
enabled_for_marketing_drip    0
org_id                        0
invited_by_user_id            0
dtype: int64

In [11]:
# Convert dates into datetime format
takehome_users_df['creation_time'] = takehome_users_df['creation_time'].apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S'))
takehome_users_df['last_session_creation_time'] = takehome_users_df['last_session_creation_time'].apply(lambda x: datetime.fromtimestamp(x))

# Convert float to int
takehome_users_df['invited_by_user_id'] = takehome_users_df['invited_by_user_id'].astype(int)


In [12]:
takehome_users_df.dtypes

object_id                              int64
creation_time                 datetime64[ns]
name                                  object
email                                 object
creation_source                       object
last_session_creation_time    datetime64[ns]
opted_in_to_mailing_list               int64
enabled_for_marketing_drip             int64
org_id                                 int64
invited_by_user_id                     int64
dtype: object

In [13]:
takehome_user_engagement_df.dtypes

time_stamp    object
user_id        int64
visited        int64
dtype: object

In [14]:
# Convert object into datetime format
takehome_user_engagement_df['time_stamp'] = takehome_user_engagement_df['time_stamp'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))


In [15]:
# Count the total number of logins for each user_id
total_logins = takehome_user_engagement_df.user_id.value_counts().reset_index()
total_logins.columns = [['user_id','freq']]
total_logins

,user_id,freq
0,3623,606
1,906,600
2,1811,593
3,7590,590
4,8068,585
...,...,...
8818,4699,1
8819,4698,1
8820,4697,1
8821,4696,1


In [16]:
# How often do logins of 1 occur?
total_logins['freq'].value_counts()

(freq,)
1          6235
2           340
3           241
4           181
5           110
           ... 
310           1
311           1
312           1
314           1
606           1
Length: 402, dtype: int64

The majority of users have only logged in once, since this frequency table shows there are 6235 users that have logged in once. The definition of an adopted user of 3 times or more is almost part of the long right tail. 

In [17]:
distinct_users = list(set(takehome_user_engagement_df['user_id']))
len(distinct_users)

8823

In [18]:
def logins_in_a_week(user_id,timestamps):
    for i in range(len(timestamps)-2):
        if (timestamps[i+2] - timestamps[i]).days <= 7:
            return 1
    return 0

In [19]:
# group by user_id and sort time_stamp column to check the number of logins across a rolling 7 day period
login_log_dict = {}

for user_id in distinct_users:
    timestamps = list(takehome_user_engagement_df['time_stamp'][takehome_user_engagement_df['user_id'] == user_id])
    if len(timestamps) >= 3:
        timestamps = sorted(timestamps)
        login_log_dict[user_id] = logins_in_a_week(user_id, timestamps)

In [20]:
adopted_user = pd.DataFrame(list(login_log_dict.items()),columns = ['user_id','adopted_user'])
adopted_user

,user_id,adopted_user
0,2,1
1,10,1
2,20,1
3,33,1
4,42,1
...,...,...
2243,11975,1
2244,11980,0
2245,11981,0
2246,11988,1


In [21]:
print(f'There are a total of {len(distinct_users)} distinct users, of which {adopted_user.shape[0]} had 3 or more logins')

There are a total of 8823 distinct users, of which 2248 had 3 or more logins


In [22]:
df_merged = takehome_users_df.join(adopted_user, how='left', lsuffix='object_id', rsuffix='user_id')
df_merged.adopted_user.fillna(value=0, inplace=True)
df_merged.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,user_id,adopted_user
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,2014-04-21 20:53:30,1,0,11,10803,2.0,1.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-30 20:45:04,0,0,1,316,10.0,1.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,2013-03-19 16:14:52,0,0,94,1525,20.0,1.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,2013-05-22 01:09:28,0,0,1,5151,33.0,1.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,2013-01-22 02:14:20,0,0,193,5240,42.0,1.0


In [23]:
print(f'{df_merged.adopted_user.mean()}% of all users are adopted users')

0.13532811968718123% of all users are adopted users


In [24]:
# Convert datetimes to days since the latest date
last_date = df_merged['creation_time'].max()
df_merged['creation_time'] = df_merged['creation_time'].apply(lambda x: (last_date-x).days)
df_merged['last_session_creation_time'] = df_merged['last_session_creation_time'].apply(lambda x: (last_date-x).days)


In [25]:
# Drop unused categorical columns
df_merged.drop(['object_id','name','email'], axis=1, inplace=True)

In [26]:
df_merged.head(5)

,creation_time,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,user_id,adopted_user
0,38,GUEST_INVITE,39,1,0,11,10803,2.0,1.0
1,196,ORG_INVITE,61,0,0,1,316,10.0,1.0
2,437,ORG_INVITE,437,0,0,94,1525,20.0,1.0
3,374,GUEST_INVITE,373,0,0,1,5151,33.0,1.0
4,498,GUEST_INVITE,493,0,0,193,5240,42.0,1.0


In [27]:
# One hot encode the creation_source column into 5 binary columns
enc = OneHotEncoder(handle_unknown='ignore')
output = enc.fit_transform(df_merged[['creation_source']])

df_merged[enc.categories_[0]] = output.toarray()
# Drop categorical column
df_merged.drop(['creation_source'], axis=1, inplace=True)

In [28]:
# Replace the invited_by_user_id column with the last_session_creation_time in days of the referral person
df_merged_all_users = df_merged.join(df_merged['last_session_creation_time'], how='left', lsuffix='invited_by_user_id', rsuffix='user_id')
df_merged_all_users.rename(columns = {'last_session_creation_timeuser_id':'last_session_creation_referral_id'}, inplace = True)

# Drop the column invited_by_user_id
df_merged_all_users.drop(['invited_by_user_id','last_session_creation_timeinvited_by_user_id'], axis=1, inplace=True)


In [29]:
df_merged_all_users.head()

,creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,user_id,adopted_user,GUEST_INVITE,ORG_INVITE,PERSONAL_PROJECTS,SIGNUP,SIGNUP_GOOGLE_AUTH,last_session_creation_referral_id
0,38,1,0,11,2.0,1.0,1.0,0.0,0.0,0.0,0.0,39
1,196,0,0,1,10.0,1.0,0.0,1.0,0.0,0.0,0.0,61
2,437,0,0,94,20.0,1.0,0.0,1.0,0.0,0.0,0.0,437
3,374,0,0,1,33.0,1.0,1.0,0.0,0.0,0.0,0.0,373
4,498,0,0,193,42.0,1.0,1.0,0.0,0.0,0.0,0.0,493


In [30]:
#Segment the dataset into users that are adopted vs. not
df_merged_all_users.groupby(by='adopted_user').mean()

,creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,user_id,GUEST_INVITE,ORG_INVITE,PERSONAL_PROJECTS,SIGNUP,SIGNUP_GOOGLE_AUTH,last_session_creation_referral_id
adopted_user,,,,,,,,,,,
0.0,315.212610,0.249705,0.150478,143.268974,5960.496568,0.177743,0.361384,0.087167,0.213003,0.160703,256.341985
1.0,319.879397,0.268844,0.161642,138.120603,6055.122278,0.194305,0.360972,0.082915,0.228643,0.133166,260.611390


Defining an "adopted user" as a user who has logged into the product on three separate days in at least one seven day period gives the following results. 

A comparison between adopted users vs. ones that are not show the following differences in features:
- Creation time of accounts for adopted users tend to be slightly longer (approx 4 days)
- Adopted users have higher adoptions to the mailing list (1.9%) and on the regular marketing email drip (1%)
- Adopted users were more likely (1.6%) to be guest invites (invited to an organization as a guest with limited permissions)
- Adopted users were less likely (2.8%) to have signed up using Google Authentication
- Referral people of adopted users were more likely to have created their own account earlier (approx 4 days)

It is important to note that only 13.5% of all users are adopted users. 
The majority of users only logged in once (70.7%) and would not count as an adopted user. 